In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# Make predictions on the test set
test_predictions = model.predict(X_test)

# Invert predictions and actual values to original scale
test_predictions = scaler.inverse_transform(test_predictions)
y_test = scaler.inverse_transform([y_test])

# Calculate evaluation metrics
mae = mean_absolute_error(y_test[0], test_predictions[:, 0])
rmse = np.sqrt(mean_squared_error(y_test[0], test_predictions[:, 0]))
print('Mean Absolute Error (MAE):', mae)
print('Root Mean Squared Error (RMSE):', rmse)

# Visualize predictions vs actual values
plt.figure(figsize=(10, 6))
plt.plot(y_test[0], label='Actual')
plt.plot(test_predictions[:, 0], label='Predicted')
plt.title('Test Set Predictions vs Actual Values')
plt.xlabel('Time Steps')
plt.ylabel('Temperature')
plt.legend()
plt.show()

In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import Adam

# Define a function to create the LSTM model
def create_lstm_model(learning_rate=0.001, units=50, dropout_rate=0.2):
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(units=units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Wrap the Keras model in a scikit-learn estimator
estimator = KerasRegressor(build_fn=create_lstm_model)

# Define the grid of hyperparameters to search
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'units': [50, 100],
    'dropout_rate': [0.2, 0.3]
}

# Perform grid search with 3-fold cross-validation
grid_search = GridSearchCV(estimator=estimator, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=1)
grid_result = grid_search.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1, verbose=0)

# Print the best hyperparameters and the corresponding mean squared error
print("Best Hyperparameters:", grid_result.best_params_)
print("Best Mean Squared Error:", -grid_result.best_score_)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Load and preprocess the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"
data = pd.read_csv(url, parse_dates=['Date'], index_col='Date')
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Define function to create dataset with input and output sequences
def create_dataset(data, time_steps):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps), 0])
        y.append(data[i + time_steps, 0])
    return np.array(X), np.array(y)

# Set time steps and split into train and test sets
time_steps = 30
X, y = create_dataset(data_scaled, time_steps)
X_train, X_test = X[:int(X.shape[0]*0.8)], X[int(X.shape[0]*0.8):]
y_train, y_test = y[:int(y.shape[0]*0.8)], y[int(y.shape[0]*0.8):]

# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1)

# Plot training loss and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Make predictions
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

# Invert predictions to original scale
train_predictions = scaler.inverse_transform(train_predictions)
y_train = scaler.inverse_transform([y_train])
test_predictions = scaler.inverse_transform(test_predictions)
y_test = scaler.inverse_transform([y_test])

# Calculate RMSE
train_rmse = np.sqrt(mean_squared_error(y_train[0], train_predictions[:,0]))
test_rmse = np.sqrt(mean_squared_error(y_test[0], test_predictions[:,0]))
print('Train RMSE:', train_rmse)
print('Test RMSE:', test_rmse)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load the dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"
data = pd.read_csv(url, parse_dates=['Date'], index_col='Date')

# Preprocessing
# Check for missing values
missing_values = data.isnull().sum()
print("Missing Values:\n", missing_values)

# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# Splitting into training and test sets
train_size = int(len(data_scaled) * 0.8)  # 80% for training
test_size = len(data_scaled) - train_size
train, test = data_scaled[0:train_size,:], data_scaled[train_size:len(data_scaled),:]

print("Training set size:", len(train))
print("Test set size:", len(test))

# Visualize the original and normalized data
plt.figure(figsize=(10, 6))
plt.plot(data.index, data['Temp'], label='Original Data')
plt.title('Daily Minimum Temperatures in Melbourne')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.legend()
plt.show()

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the model
epochs = 100
batch_size = 32
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Define number of epochs
epochs = 50

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=64, 
                    validation_data=(X_val, y_val), 
                    shuffle=True)

# Plot training/validation loss
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# Make predictions on the test set
test_predictions = model.predict(X_test)

# Invert predictions and actual values to original scale
test_predictions = scaler.inverse_transform(test_predictions)
y_test = scaler.inverse_transform([y_test])

# Calculate evaluation metrics
mae = mean_absolute_error(y_test[0], test_predictions[:, 0])
rmse = np.sqrt(mean_squared_error(y_test[0], test_predictions[:, 0]))
print('Mean Absolute Error (MAE):', mae)
print('Root Mean Squared Error (RMSE):', rmse)

# Visualize predictions vs actual values
plt.figure(figsize=(10, 6))
plt.plot(y_test[0], label='Actual')
plt.plot(test_predictions[:, 0], label='Predicted')
plt.title('Test Set Predictions vs Actual Values')
plt.xlabel('Time Steps')
plt.ylabel('Temperature')
plt.legend()
plt.show()

In [ ]:
Challenges Encountered during Model Training and Optimization:

One common challenge is overfitting, especially when dealing with complex LSTM architectures and limited training data. This can be mitigated through techniques like dropout and regularization.
Another challenge is finding the right balance between model complexity and computational resources. Increasing the number of LSTM layers or units can improve model performance, but it also increases training time and resource requirements.
Decision on Number of LSTM Layers and Units:

The decision on the number of LSTM layers and units depends on the complexity of the data and the modeling task. It often involves a trade-off between model complexity and computational resources.
Generally, starting with a simpler model and gradually increasing complexity while monitoring performance on a validation set is a common approach.
Preprocessing Steps on Time Series Data:

Preprocessing steps typically include handling missing values, normalization to bring all features to a similar scale, and splitting the data into training and test sets while preserving temporal order.
Additionally, feature engineering may involve creating lag features, extracting trend and seasonality, or encoding categorical variables.
Purpose of Dropout Layers in LSTM Networks:

Dropout layers are used in LSTM networks to prevent overfitting by randomly dropping a proportion of neurons during training. This prevents neurons from relying too heavily on specific inputs and encourages the network to learn more robust features.
Dropout helps in improving generalization by reducing the model's reliance on any single feature or combination of features, thus making the network less sensitive to noise in the training data.
Model's Ability to Capture Long-Term Dependencies and Make Accurate Predictions:

LSTM networks are specifically designed to capture long-term dependencies in sequential data. Their ability to remember information over extended time periods makes them well-suited for time series forecasting tasks.
The accuracy of predictions depends on various factors including the quality and quantity of data, the chosen architecture, hyperparameters, and training methodology.
Potential Improvements or Alternative Approaches:

Experimenting with different architectures, such as adding more LSTM layers or units, using different activation functions, or incorporating attention mechanisms, could potentially improve performance.
Ensemble methods, such as combining predictions from multiple models or incorporating external data sources, may also enhance forecasting accuracy.
Bayesian optimization or reinforcement learning-based approaches could be explored for more efficient hyperparameter tuning.
Additionally, considering alternative models such as CNNs or Transformer architectures for time series forecasting tasks could offer valuable insights.

User